# COMP 6321 Machine Learning
## Assignment 1 Answers
### Name : [Parsa Kamalipour](https://benymaxparsa.github.io) , StudentID : 40310734

In [85]:
import os

# Import libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Exercise 2: Logistic Regression (9 pts)

In this exercise you will implement the logistic regression algorithm and evaluate it on the dataset provided
with this assignment. The training dataset is divided into five different csv files. You need to combine this
into a single training dataset. Do not use any machine learning library, but feel free to use libraries for linear
algebra and feel free to verify your results with existing machine learning libraries.

## Combine the Training Data:

In [86]:
directory = "logistic_regression-dataset"
test_x = None
test_y = None
data_x = None
data_y = None

for filename in os.listdir(directory):
    if filename.__eq__('testData.csv'):
        test_x = pd.read_csv(os.path.join(directory, filename), header=None)
    elif filename.__eq__('testLabels.csv'):
        test_y = pd.read_csv(os.path.join(directory, filename), header=None,)
    else:
        for i in range(1, 6):
             if filename.__eq__(f'trainData{i}.csv'):
                 if data_x is None:
                     data_x = pd.read_csv(os.path.join(directory, filename), header=None)
                 else:
                    data_x = pd.concat([data_x, pd.read_csv(os.path.join(directory, filename), header=None)], ignore_index=True)
        for i in range(1, 6):
            if filename.__eq__(f'trainLabels{i}.csv'):
                if data_y is None:
                    data_y = pd.read_csv(os.path.join(directory, filename), header=None,)
                else:
                    data_y = pd.concat([data_y, pd.read_csv(os.path.join(directory, filename), header=None)], ignore_index=True)


In [87]:
display(test_x, test_y)

,0,1,2,3,4,5,6,7,8,9,...,54,55,56,57,58,59,60,61,62,63
0,0,0,0,1,13,10,0,0,0,8,...,16,0,0,0,4,0,14,0,15,0
1,0,0,0,0,0,7,0,0,0,0,...,16,0,0,0,0,0,2,11,12,0
2,16,11,5,16,15,16,16,16,16,16,...,14,16,16,16,16,16,16,16,16,0
3,0,0,7,16,2,2,0,0,0,1,...,11,0,0,0,0,9,9,0,0,0
4,0,0,0,7,0,0,0,0,0,0,...,0,0,0,0,0,13,14,12,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105,0,0,0,4,0,0,0,0,0,0,...,0,0,0,0,0,0,1,6,0,0
106,0,0,0,8,2,12,0,0,7,0,...,6,0,0,0,5,10,6,5,0,0
107,16,14,16,16,16,16,7,16,0,16,...,6,10,16,9,16,16,16,10,9,16
108,0,0,0,0,10,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


,0
0,6
1,6
2,5
3,6
4,6
...,...
105,6
106,5
107,5
108,6


In [88]:
display(data_x, data_y)

,0,1,2,3,4,5,6,7,8,9,...,54,55,56,57,58,59,60,61,62,63
0,0,0,11,0,16,0,0,0,4,0,...,0,0,0,0,0,0,3,5,0,0
1,0,0,9,16,16,16,0,0,0,0,...,5,0,3,0,15,13,10,3,0,16
2,9,16,15,10,16,16,0,0,16,13,...,16,16,0,16,16,7,16,16,16,3
3,0,0,0,6,0,1,0,0,0,0,...,0,0,0,0,0,0,2,0,0,0
4,0,5,0,14,16,0,0,0,0,0,...,15,0,0,3,0,9,0,8,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,0,0,0,0,4,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
496,0,0,16,16,9,15,0,0,4,7,...,0,7,0,2,14,9,13,0,0,1
497,0,0,0,6,4,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
498,1,4,10,16,13,15,12,1,7,12,...,5,2,8,0,0,5,11,0,0,0


,0
0,6
1,6
2,6
3,6
4,6
...,...
495,5
496,5
497,6
498,6


1) (2 pts) Let $\Pr(C_1|x) = \sigma({\bf{w}}^T{\bf{x}}+w_0)$ and $\Pr(C_2|{\bf{x}}) = 1 - \sigma({\bf{w}}^T{\bf{x}}+w_0)$. Learn the parameters ${\bf{w}}$ and $w_0$ using the gradient descent algorithm. Use the maximum number of epochs to be 100 for GD. You can also use any appropriate convergence criteria to stop the GD loop before 100 epochs. Use a step size of 0.1 (or 0.01 if it is converging poorly) for GD. 

## Initialize Parameters:

In [89]:
data_dimension = len(data_x.columns)
number_of_samples= data_x.shape[0]
w_weight_vector = np.array([0.0]*data_dimension)
w0_bias = 0.0
learning_rate_large = 0.1
learning_rate_small = 0.01
max_epochs = 100
previous_loss = float('inf')
epsilon_tolerance = 1e-6

## Logistic Regression using Gradient Descent Algorithm:

In [90]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

In [91]:
def function_input_calculator(w, x, w0):
    return np.dot(x, w) + w0

In [92]:
def gradient_calculator(x, err, num_of_samples):
    gradient_w = np.dot(x.T, err)/num_of_samples
    gradient_w0 = np.sum(err)/num_of_samples
    return gradient_w, gradient_w0

In [93]:
def learn(x, err, num_of_samples, learn_rate):
    gradient_of_w, gradient_of_w0 = gradient_calculator(x, err, num_of_samples)
    w = learn_rate * gradient_of_w
    w0 = learn_rate * gradient_of_w0
    return w, w0

In [94]:
def loss_function(y, predict_y):
    return -np.mean(y * np.log(predict_y + 1e-15) + (1 - y) * np.log(1 - predict_y + 1e-15))

In [95]:
loss_history = []

In [96]:
for epoch in range(max_epochs):
    z = function_input_calculator(w_weight_vector, data_x, w0_bias)
    predicted_y = sigmoid(z)
    
    error = np.subtract(predicted_y , np.array(data_y))
    
    error = error[0]
    
    w_new, w0_new = learn(data_x, error, number_of_samples, learning_rate_large)
    
    w_weight_vector -= w_new
    w0_bias -= w0_new
    
    loss = loss_function(data_y.to_numpy(), predicted_y)
    loss_history.append(loss)
    
    print(print(f'Epoch {epoch + 1}, Loss: {loss}'))
    if abs(previous_loss - loss) < epsilon_tolerance:
        print("Convergence criteria met.")
        break
    previous_loss = loss
    

Epoch 1, Loss: 0.6931471805599436
None
Epoch 2, Loss: -151.2961254385624
None
Epoch 3, Loss: -152.16755908360693
None
Epoch 4, Loss: -152.43349104686772
None
Epoch 5, Loss: -152.59984926085042
None
Epoch 6, Loss: -152.752677494821
None
Epoch 7, Loss: -152.90464952763486
None
Epoch 8, Loss: -153.03373338170894
None
Epoch 9, Loss: -153.1167111571549
None
Epoch 10, Loss: -153.17912222048867
None
Epoch 11, Loss: -153.2236475005079
None
Epoch 12, Loss: -153.25691385046076
None
Epoch 13, Loss: -153.28836550591623
None
Epoch 14, Loss: -153.31980865664354
None
Epoch 15, Loss: -153.35124663887166
None
Epoch 16, Loss: -153.38268148251407
None
Epoch 17, Loss: -153.41411442113278
None
Epoch 18, Loss: -153.44554620379
None
Epoch 19, Loss: -153.47697728513464
None
Epoch 20, Loss: -153.50840794104295
None
Epoch 21, Loss: -153.53983833888606
None
Epoch 22, Loss: -153.57126858019464
None
Epoch 23, Loss: -153.60269872655672
None
Epoch 24, Loss: -153.63412881532903
None
Epoch 25, Loss: -153.6655588691684

In [97]:
print("\nLearned weight vector w:")
print(w_weight_vector)
print("\nLearned bias term w0:")
print(w0_bias)


Learned weight vector w:
[193.2528     191.4744     275.652      380.133      362.4231
 286.4706     223.1892     180.5817     185.25       213.8526
 364.4979     388.1358     297.5856     248.0868     195.5499
 170.8005     181.6191     226.5978     372.723      288.6195
 207.1836     185.6946     174.7278     177.4695     185.3241
 232.3776037  396.2868     333.5241     273.5772     219.2619
 181.6191     176.2839     195.4758     218.2986     341.2305
 337.3773     349.90020064 310.9236     217.2612     176.7285
 181.545      207.5541     298.0302     288.1008     288.5454
 339.6744     269.1312     180.2853     179.9148     199.4772
 306.6999     333.1536     310.5531     332.709      277.5786
 187.2507     177.3213     194.9571     276.9117     374.5755
 377.2431     302.92080129 215.5569     195.1053    ]

Learned bias term w0:
37.051549844376424
